# ME Bank Policy Portal Sraper
Suse a simple login.

FOr now loggign in with the browser, then cut and past the auth cookies into this notebook.

Scraper Code
1. Only searchs for links in dic.container
2. does niot use HTML extention
   

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
from loguru import logger

def create_empty_html_document():
    # Create a new BeautifulSoup object with a basic HTML structure
    soup = BeautifulSoup('', 'html.parser')
    
    # Create the html tag
    html_tag = soup.new_tag('html')
    soup.append(html_tag)
    
    # Create the head tag
    head_tag = soup.new_tag('head')
    html_tag.append(head_tag)
    
    # Add basic elements to the head
    # title_tag = soup.new_tag('title')
    # title_tag.string = 'Document Title'
    # head_tag.append(title_tag)
    
    # Add meta charset tag
    meta_tag = soup.new_tag('meta')
    meta_tag['charset'] = 'UTF-8'
    head_tag.append(meta_tag)
    
    # Create the body tag
    body_tag = soup.new_tag('body')
    html_tag.append(body_tag)
    
    # Return the formatted HTML string
    return soup


def scrape_page_copy(heading, url, cookie):
    """
    Scrapes the content of div with class 'page-copy' from the given URL.
    
    Args:
        url (str): The URL of the webpage to scrape.
        
    Returns:
        str: The HTML content inside the div with class 'page-copy'.
             Returns None if the div is not found or if there's an error.
    """
    try:
        # Send HTTP request to the URL
        
        response = requests.get(url, 
                                headers={'User-Agent': 'Mozilla/5.0'},
                                cookies={'Cookie': cookie})
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the div with class 'page-copy'
    page_copy_div = soup.find('div', class_='page-copy')
    
    if not page_copy_div:
        print(f"{url} no 'page-copy'")
        return None

   
    # Create an empyt document and append the heading and 
    # the page-copy to the body 
    output_soup = create_empty_html_document()
    body = output_soup.body
    h2_tag = soup.new_tag('h2')
    h2_tag.string = heading
    body.append(h2_tag)
    body.append(page_copy_div)

    return output_soup


def extract_side_navigation_links(url, cookie):
    """
    Extract all links from a div with class 'side-navigation' on a webpage.
    
    Args:
        url (str): The URL of the webpage to scrape
        
    Returns:
        list: A list of dictionaries containing link text and href values
    """
    # Send HTTP request to the URL
    try:
        response = requests.get(url, 
                                headers={'User-Agent': 'Mozilla/5.0'},
                                cookies={'Cookie': cookie})
        response.raise_for_status()  # Raise exception for HTTP errors
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return []
    
    #find the list of pages in the side navigation
    soup = BeautifulSoup(response.text, 'html.parser')
    side_nav = soup.find('div', class_='side-navigation')
    
    if not side_nav:
        print("side navigation links not found")
        return []
    
    links = side_nav.find_all('a')
    
    for link in links:
        href = link.get('href')
        heading = link.get_text(strip=True)
        if href and not href.startswith(('http://', 'https://')):
            href = urllib.parse.urljoin("https://mebroker.powerappsportals.com", href)
        
        output_soup = scrape_page_copy(heading, href, cookie=cookie)
        
        path = urllib.parse.urlparse(href).path
        if path[-1] == "/":
            path = path[:-1]+".html"

        directory = "policy/MEB/ME_Broker"
        full_path =  directory + path
        directory = os.path.dirname(full_path)        
        if not os.path.exists(directory):
            os.makedirs(directory) 

        logger.info(f"Downloading {url} to {full_path}")
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(str(output_soup))
        

links = extract_side_navigation_links(r"https://mebroker.powerappsportals.com/policy-home1/",
    cookie = "Dynamics365PortalAnalytics=HN_NBiH3HbiwAvgk3ncUuXRh2XUPwRj5iENmOGc3Q6c17nxCt2n4j0J4uMv3d-W9UzRpAL_cgwAmimVKy4F9zfYxGHmzP_oEeCMyFFpu13t_3EleBOZqNWSJAY1XRZNyGIFrTK2HhBGNWGdclZWTcw2; ARRAffinity=ae5dafc3502301c5972f13d1cd832204eb53fc43da4059b5dcd299d08fa97ae7; ARRAffinitySameSite=ae5dafc3502301c5972f13d1cd832204eb53fc43da4059b5dcd299d08fa97ae7; timezoneoffset=-480; isDSTSupport=false; isDSTObserved=false; ContextLanguageCode=en-US; ai_user=CmX+zhgE0lwAt+nsYL+VBY|2025-03-13T05:52:43.242Z; timeZoneCode=210; __RequestVerificationToken=2AJ0H3uTbQdP57mB-jMzxPgzjlsEzyUfYsjUNJxi9GxjUT08SxyogsqCvWz1MQ39EFvYV-81TemrH311-s-LKt9FKzkXBB2pbXex2LHSSHI1; OpenIdConnect.nonce.vx3x4IPCPBYmfpD%2BckIpCOVurQ6G67w%2Fw5BrqiWLP2E%3D=RHdEVjNiTnhkT0pUcUJfUFJzX3RZaDFySUtkX2dUaDFuRmhTZEZ5WEVKS3B4VjAxZFkwU2txR3g4Uk5JMmNQR2VDWVRZSHgwUEVpNkg0Y0FBcUtSVTkzdXc4MFhSRnhLdi1pNHJtbXpQZURYa0ZleWttR3NHYmlrVHJjRTdTV29wRXdvMVo1NUt3ak9EMlRrV18tVmthb1RKbl8wcndBZjZuR083SHhVak43Wmo2VjJydzl6QU1RYmRTWWJzWHhUVVZnS0pCaktJSUQwMWM3OGVkemNmd3l2dkhGaWF2Z0N4c2xwbXFUeWJJTQ%3D%3D; ai_session=qpxbqYvLJYkSzyvE0VTVDg|1741845163347|1741846476119; .AspNet.ApplicationCookie=JZxM41S2gKCsHTmun7PBOIJSRGqwtVWwen0zx78L4QERjpLB1S82E56s-fqX2e3ZkqO_gsvxpTOgnmP9BXUTYrzPuQJd2ei4YCwARc54Hq2087AffhE481vRqEqxItJhlPgrcrtRLKmvk5Gl-fF9-l60k8z4wD0495bG89a_kKo6KkmdA9a4Tc1HnqJYk0kS1aqvhfikVvu09hHvvuQtE7faI2XvMmVVlSxm_WRMYPY86J53SL_m9X4G-tV2qHlPbD-QmIyc0y7iO-foR9yWm3U5x9So1bJHfOktDgEnNmJn3koop2U4WWQD1Bwv-P64QAQd4_bfbKYh5Q-GRmgzt5xTYFVvBIYJUWRsncD4PFYe7nsBumTLk5oPDXsn0i99a8s3K_koQEu-fVwUf0L60rCMwEa6g8n_RNzK_gu_MiSVB8fVtqK76HgNgmx2IcZ0qvrrXyIX1dUVZ2f94o7njYYzsY2DKnsGZMrEGB_AP56_FIL8_4uHx-87hh9Wro7nQh3apUjMl3T0PH4CINpWMQ8WmQLfQhX6vRTd_Rp2A840UMM6-KtfDl6DfDV72NSVm3DaqMRuouvtGplJumGNxg")
links

# html_text = scrape_page_copy(
#     heading = "1.6 Our obligations",
#     url = r"https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/about-this-policy/our-obligations/",
#     cookie = "Dynamics365PortalAnalytics=HN_NBiH3HbiwAvgk3ncUuXRh2XUPwRj5iENmOGc3Q6c17nxCt2n4j0J4uMv3d-W9UzRpAL_cgwAmimVKy4F9zfYxGHmzP_oEeCMyFFpu13t_3EleBOZqNWSJAY1XRZNyGIFrTK2HhBGNWGdclZWTcw2; ARRAffinity=ae5dafc3502301c5972f13d1cd832204eb53fc43da4059b5dcd299d08fa97ae7; ARRAffinitySameSite=ae5dafc3502301c5972f13d1cd832204eb53fc43da4059b5dcd299d08fa97ae7; timezoneoffset=-480; isDSTSupport=false; isDSTObserved=false; ContextLanguageCode=en-US; ai_user=CmX+zhgE0lwAt+nsYL+VBY|2025-03-13T05:52:43.242Z; timeZoneCode=210; __RequestVerificationToken=2AJ0H3uTbQdP57mB-jMzxPgzjlsEzyUfYsjUNJxi9GxjUT08SxyogsqCvWz1MQ39EFvYV-81TemrH311-s-LKt9FKzkXBB2pbXex2LHSSHI1; OpenIdConnect.nonce.vx3x4IPCPBYmfpD%2BckIpCOVurQ6G67w%2Fw5BrqiWLP2E%3D=RHdEVjNiTnhkT0pUcUJfUFJzX3RZaDFySUtkX2dUaDFuRmhTZEZ5WEVKS3B4VjAxZFkwU2txR3g4Uk5JMmNQR2VDWVRZSHgwUEVpNkg0Y0FBcUtSVTkzdXc4MFhSRnhLdi1pNHJtbXpQZURYa0ZleWttR3NHYmlrVHJjRTdTV29wRXdvMVo1NUt3ak9EMlRrV18tVmthb1RKbl8wcndBZjZuR083SHhVak43Wmo2VjJydzl6QU1RYmRTWWJzWHhUVVZnS0pCaktJSUQwMWM3OGVkemNmd3l2dkhGaWF2Z0N4c2xwbXFUeWJJTQ%3D%3D; ai_session=qpxbqYvLJYkSzyvE0VTVDg|1741845163347|1741846476119; .AspNet.ApplicationCookie=JZxM41S2gKCsHTmun7PBOIJSRGqwtVWwen0zx78L4QERjpLB1S82E56s-fqX2e3ZkqO_gsvxpTOgnmP9BXUTYrzPuQJd2ei4YCwARc54Hq2087AffhE481vRqEqxItJhlPgrcrtRLKmvk5Gl-fF9-l60k8z4wD0495bG89a_kKo6KkmdA9a4Tc1HnqJYk0kS1aqvhfikVvu09hHvvuQtE7faI2XvMmVVlSxm_WRMYPY86J53SL_m9X4G-tV2qHlPbD-QmIyc0y7iO-foR9yWm3U5x9So1bJHfOktDgEnNmJn3koop2U4WWQD1Bwv-P64QAQd4_bfbKYh5Q-GRmgzt5xTYFVvBIYJUWRsncD4PFYe7nsBumTLk5oPDXsn0i99a8s3K_koQEu-fVwUf0L60rCMwEa6g8n_RNzK_gu_MiSVB8fVtqK76HgNgmx2IcZ0qvrrXyIX1dUVZ2f94o7njYYzsY2DKnsGZMrEGB_AP56_FIL8_4uHx-87hh9Wro7nQh3apUjMl3T0PH4CINpWMQ8WmQLfQhX6vRTd_Rp2A840UMM6-KtfDl6DfDV72NSVm3DaqMRuouvtGplJumGNxg"
# )

# html_text

2025-03-13 15:00:12.902 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1.html
2025-03-13 15:00:13.991 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy.html
2025-03-13 15:00:14.940 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/about-this-policy.html


https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/about-this-policy/ no 'page-copy'


2025-03-13 15:00:15.957 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/about-this-policy/definition.html
2025-03-13 15:00:16.932 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/about-this-policy/scope.html
2025-03-13 15:00:17.884 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/about-this-policy/purpose.html
2025-03-13 15:00:18.838 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/about-this-policy/related-documents.html
2025-03-13 15:00:20.168 | INFO     | __main__:extract

https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/who-can-borrow-from-us/ no 'page-copy'


2025-03-13 15:00:24.741 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/who-can-borrow-from-us/who-we-lend-to.html
2025-03-13 15:00:25.717 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/who-can-borrow-from-us/who-we-dont-lend-to.html
2025-03-13 15:00:26.762 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/who-can-borrow-from-us/joint-borrowers.html
2025-03-13 15:00:27.787 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/home-lending-policy/who-can-borrow-from-us/determining-a-benefit.html
202

...

2025-03-13 15:01:28.066 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/about-this-guide/a.1.-what-are-we-validating.html
2025-03-13 15:01:29.151 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/about-this-guide/a.2-general-rules.html
2025-03-13 15:01:30.218 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/citizenship-residency-and-visas.html


https://mebroker.powerappsportals.com/policy-home1/verification-and-validation-procedures/citizenship-residency-and-visas/ no 'page-copy'


2025-03-13 15:01:31.311 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/citizenship-residency-and-visas/b.1.-validating-eligible-temporary-residents.html
2025-03-13 15:01:32.285 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/citizenship-residency-and-visas/b.2.-acceptable-temporary-visa-types.html
2025-03-13 15:01:33.335 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/payslip-and-account-statement-requirements.html
2025-03-13 15:01:34.447 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.co

https://mebroker.powerappsportals.com/policy-home1/verification-and-validation-procedures/verifying-commitments/ no 'page-copy'


2025-03-13 15:01:44.778 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/verifying-commitments/living-expenses.html
2025-03-13 15:01:45.825 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/verifying-commitments/other-debts.html
2025-03-13 15:01:46.796 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validation-procedures/verifying-commitments/funds-to-complete.html
2025-03-13 15:01:47.945 | INFO     | __main__:extract_side_navigation_links:133 - Downloading https://mebroker.powerappsportals.com/policy-home1/ to policy/MEB/ME_Broker/policy-home1/verification-and-validat